## Kaggle Titanic
１．欠損値の補完  
 train :  
   Age : median　中央値  
   Embarked : 最頻値  "S"
 test :  
   Age : median　中央値  
   Fare : median　中央値  
２．カテゴリカル変数をダミー変数に変換  
  Sex, Pclass, Embarked  
３．不要な列の削除  
 train : 'PassengerId', 'Name', 'Ticket', 'Cabin'  
 tset  : 'Name', 'Ticket', 'Cabin'  
４．列追加
 Age_bin : 年齢を 10 で割ったもの  
 Fare_bin : 運賃を 25 で割ったもの  

In [108]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.utils import np_utils

%matplotlib inline

import check_miss_value

In [109]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [110]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [111]:
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [112]:
train.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.036847,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.077221,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.369226,0.083081,0.018443,-0.549500
Age,0.036847,-0.077221,-0.369226,1.000000,-0.308247,-0.189119,0.096067
SibSp,-0.057527,-0.035322,0.083081,-0.308247,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.189119,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.096067,0.159651,0.216225,1.000000


In [113]:
train.Sex.values.reshape(-1, 1).shape

(891, 1)

In [114]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [115]:
from sklearn.preprocessing import LabelEncoder

In [116]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [117]:
# LE1 = LabelEncoder()

In [118]:
# train.Sex = LE1.fit_transform(train.Sex.values.reshape(-1, 1))

In [119]:
train['Sex'].value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [120]:
train.Sex.value_counts()

male      577
female    314
Name: Sex, dtype: int64

In [121]:
# train.Ticket.value_counts()

In [122]:
# Ticket は後で削除する

In [123]:
train.Embarked.value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [124]:
# LE2 = LabelEncoder()

In [125]:
from sklearn.preprocessing import Imputer

In [126]:
# impubter = Imputer(missing_values=np.nan, strategy='most_frequent')

In [127]:
# 文字列型は Imputerで入れられないので、最頻値を直接入れる
train.Embarked = train.Embarked.fillna('S')

In [128]:
train.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [129]:
# train.Embarked = LE2.fit_transform(train.Embarked.values.reshape(-1, 1))

In [130]:
train.Embarked.value_counts()

S    646
C    168
Q     77
Name: Embarked, dtype: int64

In [131]:
train.corr()[train.corr() > 0.2]

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,1.000000,NaN,NaN,NaN,NaN,0.257307
Pclass,NaN,NaN,1.0,NaN,NaN,NaN,NaN
Age,NaN,NaN,NaN,1.0,NaN,NaN,NaN
SibSp,NaN,NaN,NaN,NaN,1.000000,0.414838,NaN
Parch,NaN,NaN,NaN,NaN,0.414838,1.000000,0.216225
Fare,NaN,0.257307,NaN,NaN,NaN,0.216225,1.000000


In [132]:
train.corr()[train.corr() < -0.2]

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Survived,NaN,NaN,-0.338481,NaN,NaN,NaN,NaN
Pclass,NaN,-0.338481,NaN,-0.369226,NaN,NaN,-0.5495
Age,NaN,NaN,-0.369226,NaN,-0.308247,NaN,NaN
SibSp,NaN,NaN,NaN,-0.308247,NaN,NaN,NaN
Parch,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,NaN,NaN,-0.549500,NaN,NaN,NaN,NaN


In [133]:
# PassengerId, Name, Ticket, Cabin　を削除
train = train.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

In [134]:
check_miss_value.check_miss_value(train)

,欠損数,%
Survived,0,0.00000
Pclass,0,0.00000
Sex,0,0.00000
Age,177,19.86532
SibSp,0,0.00000
Parch,0,0.00000
Fare,0,0.00000
Embarked,0,0.00000


In [135]:
train.Age = train.Age.fillna(train.Age.median())

In [136]:
check_miss_value.check_miss_value(train)

,欠損数,%
Survived,0,0.0
Pclass,0,0.0
Sex,0,0.0
Age,0,0.0
SibSp,0,0.0
Parch,0,0.0
Fare,0,0.0
Embarked,0,0.0


In [137]:
# testデータ
check_miss_value.check_miss_value(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,86,20.574163
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,1,0.239234
Cabin,327,78.229665


In [138]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Name           418 non-null object
Sex            418 non-null object
Age            332 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Ticket         418 non-null object
Fare           417 non-null float64
Cabin          91 non-null object
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [139]:
test.Age.median()

27.0

In [140]:
test.Age = test.Age.fillna(test.Age.median())

In [141]:
test.Fare = test.Fare.fillna(test.Fare.median())

In [142]:
check_miss_value.check_miss_value(test)

,欠損数,%
PassengerId,0,0.000000
Pclass,0,0.000000
Name,0,0.000000
Sex,0,0.000000
Age,0,0.000000
SibSp,0,0.000000
Parch,0,0.000000
Ticket,0,0.000000
Fare,0,0.000000
Cabin,327,78.229665


In [143]:
# Name, Ticket, Cabin　を削除
test = test.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [144]:
# test.Sex = LE1.fit_transform(test.Sex.values.reshape(-1, 1))

In [145]:
# test.Embarked = LE2.fit_transform(test.Sex.values.reshape(-1, 1))

In [146]:
test.Sex.value_counts()

male      266
female    152
Name: Sex, dtype: int64

In [147]:
test.Embarked.value_counts()

S    270
C    102
Q     46
Name: Embarked, dtype: int64

In [148]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


In [149]:
test.corr()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.026751,-0.031447,0.003818,0.043080,0.008633
Pclass,-0.026751,1.000000,-0.467853,0.001087,0.018721,-0.577313
Age,-0.031447,-0.467853,1.000000,-0.071197,-0.043731,0.342357
SibSp,0.003818,0.001087,-0.071197,1.000000,0.306895,0.171912
Parch,0.043080,0.018721,-0.043731,0.306895,1.000000,0.230325
Fare,0.008633,-0.577313,0.342357,0.171912,0.230325,1.000000


In [150]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 8 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         891 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Embarked    891 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 55.8+ KB


In [151]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 8 columns):
PassengerId    418 non-null int64
Pclass         418 non-null int64
Sex            418 non-null object
Age            418 non-null float64
SibSp          418 non-null int64
Parch          418 non-null int64
Fare           418 non-null float64
Embarked       418 non-null object
dtypes: float64(2), int64(4), object(2)
memory usage: 26.2+ KB


In [153]:
train.Age.min(), train.Age.max(), test.Age.min(), test.Age.max()

(0.42, 80.0, 0.17, 76.0)

In [154]:
# 年齢を 10 最刻みのレンジにする
train['Age_bin'] = train['Age'] // 10
test['Age_bin'] = test['Age'] // 10

In [155]:
train.Fare.min(), train.Fare.max(), test.Fare.min(), test.Fare.max()

(0.0, 512.3292, 0.0, 512.3292)

In [156]:
# 運賃を 20 $ 毎のレンジにする
train['Fare_bin'] = train['Fare'] // 25
test['Fare_bin'] = test['Fare'] // 25

In [157]:
# カテゴリカルを変換
train = pd.get_dummies(train, columns=['Sex', 'Embarked', 'Pclass'])

In [158]:
test = pd.get_dummies(test, columns=['Sex', 'Embarked', 'Pclass'])

In [159]:
# picle ファイルに出力
train.to_pickle('pd_train.pk2')
test.to_pickle('pd_test.pk2')

In [160]:
train.corr()

,Survived,Age,SibSp,Parch,Fare,Age_bin,Fare_bin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
Survived,1.000000,-0.064910,-0.035322,0.081629,0.257307,-0.050972,0.254902,0.543351,-0.543351,0.168240,0.003650,-0.149683,0.285904,0.093349,-0.322308
Age,-0.064910,1.000000,-0.233296,-0.172482,0.096688,0.975537,0.114402,-0.081163,0.081163,0.030248,-0.031415,-0.006729,0.323896,0.015831,-0.291955
SibSp,-0.035322,-0.233296,1.000000,0.414838,0.159651,-0.226973,0.132968,0.114631,-0.114631,-0.059528,-0.026354,0.068734,-0.054582,-0.055932,0.092548
Parch,0.081629,-0.172482,0.414838,1.000000,0.216225,-0.158376,0.193157,0.245489,-0.245489,-0.011069,-0.081228,0.060814,-0.017633,-0.000734,0.015790
Fare,0.257307,0.096688,0.159651,0.216225,1.000000,0.099775,0.993316,0.182333,-0.182333,0.269335,-0.117216,-0.162184,0.591711,-0.118557,-0.413333
Age_bin,-0.050972,0.975537,-0.226973,-0.158376,0.099775,1.000000,0.115774,-0.074794,0.074794,0.030254,-0.063081,0.013193,0.315096,0.033869,-0.299053
Fare_bin,0.254902,0.114402,0.132968,0.193157,0.993316,0.115774,1.000000,0.168488,-0.168488,0.260966,-0.118362,-0.154132,0.616042,-0.128091,-0.426537
Sex_female,0.543351,-0.081163,0.114631,0.245489,0.182333,-0.074794,0.168488,1.000000,-1.000000,0.082853,0.074115,-0.119224,0.098013,0.064746,-0.137143
Sex_male,-0.543351,0.081163,-0.114631,-0.245489,-0.182333,0.074794,-0.168488,-1.000000,1.000000,-0.082853,-0.074115,0.119224,-0.098013,-0.064746,0.137143
Embarked_C,0.168240,0.030248,-0.059528,-0.011069,0.269335,0.030254,0.260966,0.082853,-0.082853,1.000000,-0.148258,-0.782742,0.296423,-0.125416,-0.153329


In [161]:
train.corr()[train.corr() > 0.2]

,Survived,Age,SibSp,Parch,Fare,Age_bin,Fare_bin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
Survived,1.000000,NaN,NaN,NaN,0.257307,NaN,0.254902,0.543351,NaN,NaN,NaN,NaN,0.285904,NaN,NaN
Age,NaN,1.000000,NaN,NaN,NaN,0.975537,NaN,NaN,NaN,NaN,NaN,NaN,0.323896,NaN,NaN
SibSp,NaN,NaN,1.000000,0.414838,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parch,NaN,NaN,0.414838,1.000000,0.216225,NaN,NaN,0.245489,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Fare,0.257307,NaN,NaN,0.216225,1.000000,NaN,0.993316,NaN,NaN,0.269335,NaN,NaN,0.591711,NaN,NaN
Age_bin,NaN,0.975537,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,0.315096,NaN,NaN
Fare_bin,0.254902,NaN,NaN,NaN,0.993316,NaN,1.000000,NaN,NaN,0.260966,NaN,NaN,0.616042,NaN,NaN
Sex_female,0.543351,NaN,NaN,0.245489,NaN,NaN,NaN,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sex_male,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
Embarked_C,NaN,NaN,NaN,NaN,0.269335,NaN,0.260966,NaN,NaN,1.000000,NaN,NaN,0.296423,NaN,NaN


In [162]:
train.corr()[train.corr() < -0.2]

,Survived,Age,SibSp,Parch,Fare,Age_bin,Fare_bin,Sex_female,Sex_male,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
Survived,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.543351,NaN,NaN,NaN,NaN,NaN,-0.322308
Age,NaN,NaN,-0.233296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.291955
SibSp,NaN,-0.233296,NaN,NaN,NaN,-0.226973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Parch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.245489,NaN,NaN,NaN,NaN,NaN,NaN
Fare,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.413333
Age_bin,NaN,NaN,-0.226973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.299053
Fare_bin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.426537
Sex_female,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN,NaN,NaN
Sex_male,-0.543351,NaN,NaN,-0.245489,NaN,NaN,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Embarked_C,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.782742,NaN,NaN,NaN


In [169]:
train.corr()[:][['Survived', 'Sex_female', 'Sex_male', 'Pclass_1', 'Pclass_2', 'Pclass_3']]

,Survived,Sex_female,Sex_male,Pclass_1,Pclass_2,Pclass_3
Survived,1.000000,0.543351,-0.543351,0.285904,0.093349,-0.322308
Age,-0.064910,-0.081163,0.081163,0.323896,0.015831,-0.291955
SibSp,-0.035322,0.114631,-0.114631,-0.054582,-0.055932,0.092548
Parch,0.081629,0.245489,-0.245489,-0.017633,-0.000734,0.015790
Fare,0.257307,0.182333,-0.182333,0.591711,-0.118557,-0.413333
Age_bin,-0.050972,-0.074794,0.074794,0.315096,0.033869,-0.299053
Fare_bin,0.254902,0.168488,-0.168488,0.616042,-0.128091,-0.426537
Sex_female,0.543351,1.000000,-1.000000,0.098013,0.064746,-0.137143
Sex_male,-0.543351,-1.000000,1.000000,-0.098013,-0.064746,0.137143
Embarked_C,0.168240,0.082853,-0.082853,0.296423,-0.125416,-0.153329
